In [1]:
!pip install flask_sqlalchemy


In [2]:
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

class Book(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(255), nullable=False)
    author = db.Column(db.String(255), nullable=False)
    stock = db.Column(db.Integer, nullable=False)

class Member(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(255), nullable=False)
    outstanding_debt = db.Column(db.Float, nullable=False, default=0.0)

class Transaction(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    book_id = db.Column(db.Integer, db.ForeignKey('book.id'), nullable=False)
    member_id = db.Column(db.Integer, db.ForeignKey('member.id'), nullable=False)
    issue_date = db.Column(db.Date, nullable=False)
    return_date = db.Column(db.Date)
    rent_fee = db.Column(db.Float, nullable=False, default=0.0)


In [3]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Books routes
@app.route('/books', methods=['GET'])
def get_books():
    books = Book.query.all()
    return jsonify([book.serialize() for book in books])

@app.route('/books', methods=['POST'])
def create_book():
    data = request.json
    book = Book(title=data['title'], author=data['author'], stock=data['stock'])
    db.session.add(book)
    db.session.commit()
    return jsonify(book.serialize()), 201

# Members routes
@app.route('/members', methods=['GET'])
def get_members():
    members = Member.query.all()
    return jsonify([member.serialize() for member in members])

@app.route('/members', methods=['POST'])
def create_member():
    data = request.json
    member = Member(name=data['name'])
    db.session.add(member)
    db.session.commit()
    return jsonify(member.serialize()), 201


In [4]:
@app.route('/transactions', methods=['POST'])
def issue_book():
    data = request.json
    book_id = data['book_id']
    member_id = data['member_id']

    # Check if the book is available
   


In [ ]:
import requests
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/import_books', methods=['POST'])
def import_books():
    api_url = 'https://frappe.io/api/method/frappe-library'
    page = request.json.get('page', 1)
    title = request.json.get('title', '')

    params = {
        'page': page,
        'title': title
    }

    response = requests.get(api_url, params=params)
    data = response.json()

    if 'message' in data:
        books = data['message']
        for book in books:
            # Extract relevant information from the book data
            book_id = book['bookID']
            title = book['title']
            authors = book['authors']
            isbn = book['isbn']
            # ... extract other fields as needed

            # Create a book record in your database
            book = Book(book_id=book_id, title=title, authors=authors, isbn=isbn)
            db.session.add(book)
        db.session.commit()

        return jsonify({'message': 'Books imported successfully'})

    return jsonify({'message': 'Error importing books'})

if __name__ == '__main__':
    app.run()


In [ ]:
@app.route('/books')
def display_books():
    books = Book.query.all()
    return render_template('books.html', books=books)


In [ ]:
@app.route('/books/add', methods=['GET', 'POST'])
def add_book():
    if request.method == 'POST':
        # Retrieve the book details from the request form
        title = request.form['title']
        author = request.form['author']
        # ... retrieve other fields as needed

        # Create a new book record
        book = Book(title=title, author=author)
        db.session.add(book)
        db.session.commit()

        return redirect(url_for('display_books'))
    
    return render_template('add_book.html')

@app.route('/books/edit/<int:book_id>', methods=['GET', 'POST'])
def edit_book(book_id):
    book = Book.query.get_or_404(book_id)

    if request.method == 'POST':
        # Update the book details
        book.title = request.form['title']
        book.author = request.form['author']
        # ... update other fields as needed

        db.session.commit()
        return redirect(url_for('display_books'))

    return render_template('edit_book.html', book=book)

@app.route('/books/delete/<int:book_id>', methods=['POST'])
def delete_book(book_id):
    book = Book.query.get_or_404(book_id)
    db.session.delete(book)
    db.session.commit()
    return redirect(url_for('display_books'))

